In [ ]:
# Deleting blobs

from azure.storage.blob import ContainerClient
from etl_tools import load_oedi_sas

sas_token = load_oedi_sas() # Loads oedi rw sas token
client = ContainerClient.from_container_url(f'https://nrel.blob.core.windows.net/oedi?{sas_token}')
for blob in client.list_blobs():
    if "wtk" in blob.name and 'test' in blob.name:
        print(blob.name)
        #client.delete_blob(blob)

In [ ]:
# Copying blobs within Azure

source_blob = client.get_blob_client('wtk/wtk_bangladesh_hourly_ref.json')
dest_blob = client.get_blob_client('wtk/bangladesh/kerchunk_hourly_ref.json')
dest_blob.start_copy_from_url(source_blob.url)

In [ ]:
# Copy objects from S3 into Azure BLOB storage

import s3fs
from azure.storage.blob import ContainerClient
from etl_tools import load_oedi_sas

def copy_file_to_azure(source, dest, sas=None, container='oedi'):
    s3 = s3fs.S3FileSystem()
    if not sas:
        sas = load_oedi_sas()

    client = ContainerClient.from_container_url(f'https://nrel.blob.core.windows.net/{container}?{sas}')
    blob = client.get_blob_client(dest)
    with s3.open(source, 'rb') as f:
        blob.upload_blob(f.read())

copy_file_to_azure('s3://kerchunk-staging/test.txt', 'wtk/test/test.txt')